In [1]:
import numpy as np
import pandas as pd
import requests
import json
import time
import datetime
import matplotlib.pyplot as plt

In [ ]:
# Wrangle data from cryptocompare API

def blocktotime(symbol):

  # Blockchain data url
  url = 'https://min-api.cryptocompare.com/data/blockchain/histo/day?api_key=be5138d4832a7bc23f39c518f6a7e07a76cbea93ca224491190683c6d0a19ea6&limit=2000&fsym=' + symbol 
  APIKEY = 'be5138d4832a7bc23f39c518f6a7e07a76cbea93ca224491190683c6d0a19ea6'
  response = requests.get(url).json()['Data']['Data']
  df = pd.DataFrame(response)
  df['datetime'] = pd.to_datetime(df['time'], unit='s')

  # Calculate something, I will do block size to block time (b/s)
  df['sizetotime'] = df['block_size'] / df['block_time']

  # Price data url
  price_url = 'https://min-api.cryptocompare.com/data/v2/histoday?tsym=USD&limit=2000&api_key=be5138d4832a7bc23f39c518f6a7e07a76cbea93ca224491190683c6d0a19ea6&fsym=' + symbol
  price_data = requests.get(url).json()['Data']['Data']

  # Data frame datetime conversion
  price_df = pd.DataFrame(price_data)
  price_df['datetime'] = pd.to_datetime(price_df['time'], unit='s')
  prices_df = df.set_index('datetime').merge(price_df.set_index('datetime'), on='datetime')
  return prices_df


# Iterate over list
crypto_list = ['BTC', 'ETH', 'DOGE', 'LTC', 'ZEC']
bts_df = pd.DataFrame()
metric = 'sizetotime'

for c in crypto_list:
  df = blocktotime(c)
  bts_df[c] = df[metric]

bts_df

plt.figure(figsize=(18.5, 12.5))
plt.plot(bts_df.rolling(window=30).mean(), lw=1.75)     # Smoothing param
plt.title('Block Processing Speed (Block Size to Block Time Ratios (Rolling 30-day mean))', fontsize=30)
plt.xlabel("Date", fontsize=20)
plt.ylabel("BS:BT (bits per second)", fontsize=20)
plt.legend(loc=0, labels=crypto_list)